In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_visible_devices([gpus[1]], 'GPU')

sys.path.append("/nfs/scistore12/chlgrp/vvolhejn/smooth")

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/0225_gp1/")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
%%time
dataset = smooth.datasets.GaussianProcessDataset(samples_train=100, lengthscale=1.0, seed=123, dim=1000)

In [ ]:
# dataset.gp_model.plot()

In [ ]:
_history_cb = smooth.callbacks.WeightsHistoryCallback(min_snapshots=200)
_model = smooth.model.train_shallow(
    dataset,
    learning_rate=0.01,
    init_scale=100.,
    epochs=100000,
    hidden_size=500,
    batch_size=len(dataset.x_train),
    verbose=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping("loss", min_delta=1e-5, patience=1000),
        smooth.callbacks.Tqdm(verbose=0),
        tf.keras.callbacks.TerminateOnNaN(),
        _history_cb
    ],
    train_val_split=1.0,
    activation="relu",
)
# smooth.analysis.plot_shallow(_model, dataset)
# _ani = smooth.analysis.plot_shallow(_model, dataset, weights_history=_history_cb.weights_history)
# display(HTML(_ani.to_html5_video()))

In [ ]:
np.isfinite(_model.history.history["loss"][-1])

In [ ]:
_model.evaluate(dataset.x_train, dataset.y_train)

In [ ]:
k = 100
xs, ys = np.linspace(-2, 2, k), np.linspace(-2, 2, k)
grid = [(x, y) for x in xs for y in ys]
grid = np.array(grid)
# plt.contour(xs, ys, _model.predict(grid).reshape(k, k))
plt.contour(xs, ys, dataset.gp_model.predict_noiseless(grid)[0].reshape(k, k))
# grid
# plt.contour(dataset.x_test[:,0], dataset.x_test[:,1], _model.predict(dataset.x_test))
# _model.predict(grid).reshape(50, 50)
grid[:,0].shape

In [ ]:
dataset.gp_model.predict_noiseless(grid)[0]

In [ ]:
import GPy

dim = 2
gp_model = GPy.models.GPRegression(
    # It seems the constructor needs at least 1 data point.
    np.array([[0] * dim]),
    np.array([[0]]),
    noise_var=0.001,
)
gp_model.kern.lengthscale = 0.3
samples_test = 100
x_test = np.random.randn(samples_test, dim)
y_test = gp_model.posterior_samples_f(x_test, size=1)[:, :, 0]
gp_model.set_XY(x_test, y_test)
samples_train = 10
indices = smooth.util.subsample_regularly(samples_test, samples_train)
x_train = x_test[indices]
y_train = gp_model.posterior_samples_f(x_train, size=1)[:, :, 0]
y_train